In [1]:
import pandas as pd
from great_tables import GT, loc, style


# Creating a table display from Paris 2024 Olympic Medals. 

## Preparing the data

Before creating the display table, we process the DataFrame.  We get Paris 2024 data, and we get the total, gold, silver and bronze medals by Committee and discipline.

In [2]:
df_olympic = pd.read_csv("./input/olympic_medals.csv")
df_paris_2024 = df_olympic[df_olympic.Olympiad=="Paris 2024"]
df_paris_2024.head(2)

,Olympiad,Discipline,Event,Winner,Medal_type,Olympic_city,Olympic_year,Olympic_season,Gender,Code,Committee,Committee_type
20433,Paris 2024,Athletics,Women's 800m,Tsige Duguma,Silver,Paris,2024,summer,Women,ETH,Ethiopia,Country
20434,Paris 2024,Athletics,"Men's 10,000m",Berihu Aregawi,Silver,Paris,2024,summer,Men,ETH,Ethiopia,Country


In [3]:
medal_counts = (
    df_paris_2024.groupby(["Committee", "Code","Medal_type"])    
    .size()
    .unstack(fill_value=0) # Turn Medal_type into columns
    .reset_index()
)

medal_counts["Total"] = medal_counts[["Gold", "Silver", "Bronze"]].sum(axis=1)
medal_counts.sample(3)

Medal_type,Committee,Code,Bronze,Gold,Silver,Total
68,Poland,POL,5,1,4,10
78,South Africa,RSA,2,1,3,6
70,Puerto Rico,PUR,2,0,0,2


In [4]:
# Sort by Discipline, then by Gold → Silver → Bronze (descending)
medal_counts = medal_counts.sort_values(
    by=["Gold", "Silver", "Bronze"],
    ascending=[False, False, False]
)
medal_counts.head(3)

Medal_type,Committee,Code,Bronze,Gold,Silver,Total
89,United States,USA,42,40,44,126
15,China,CHN,24,40,27,91
49,Japan,JPN,13,20,12,45


In [5]:
# Group also by Gender to get medal counts per gender
gender_counts = (
    df_paris_2024[df_paris_2024["Medal_type"].isin(["Gold", "Silver", "Bronze"])]
    .groupby(["Committee", "Gender", "Medal_type"])
    .size()
    .unstack("Medal_type", fill_value=0)
    .reindex(columns=["Gold", "Silver", "Bronze"], fill_value=0)
)

In [6]:
gender_flat = (
    gender_counts
    .stack(level=0) # Stack Gender
    .unstack("Gender") # Gender to columns
)


In [7]:
gender_flat

Gender                 Men  Mixed  Open  Women
Committee  Medal_type                         
Albania    Gold        0.0    NaN   NaN    NaN
           Silver      0.0    NaN   NaN    NaN
           Bronze      2.0    NaN   NaN    NaN
Algeria    Gold        0.0    NaN   NaN    2.0
           Silver      0.0    NaN   NaN    0.0
...                    ...    ...   ...    ...
Uzbekistan Silver      1.0    NaN   NaN    1.0
           Bronze      3.0    NaN   NaN    0.0
Zambia     Gold        0.0    NaN   NaN    NaN
           Silver      0.0    NaN   NaN    NaN
           Bronze      1.0    NaN   NaN    NaN

[276 rows x 4 columns]

In [8]:
gender_flat = (
    gender_counts
    .unstack("Gender", fill_value=0)  # Gender becomes outer column level
)

In [9]:
gender_flat.columns = [f"{medal}_{gender}" for medal, gender in gender_flat.columns]
gender_flat.reset_index(inplace=True)

In [10]:
gender_flat = gender_flat.reset_index(drop=True)
# Merge with your existing medal_counts
medal_counts = medal_counts.merge(gender_flat, on=["Committee"], how="left")

In [11]:
df_flags = pd.read_csv("./input/flags.csv")
df_flags.head(2)

,Committee,Code,Flag
0,Albania,ALB,🇦🇱
1,Algeria,ALG,🇩🇿


In [12]:
medal_counts = medal_counts.merge(df_flags[["Code", "Flag"]], on="Code")

In [13]:
medal_counts["Committee"] = medal_counts["Flag"] + " " + medal_counts["Committee"] 

In [14]:
medal_counts = medal_counts.drop(columns=["Code", "Flag"])

In [15]:
proper_order = [
    "Committee", "Total", "Gold", "Silver", "Bronze",  
    "Gold_Men", "Silver_Men", "Bronze_Men", 
    "Gold_Women", "Silver_Women", "Bronze_Women",
    "Gold_Mixed", "Silver_Mixed", "Bronze_Mixed",
    "Gold_Open", "Silver_Open", "Bronze_Open"
]

# Reorder the dataframe
medal_counts = medal_counts[proper_order]


In [16]:
table = (
    GT(medal_counts.head(10), rowname_col="Committee")
    .tab_header(title="Paris 2024 Medals - Top 10", subtitle="Medals by Committee | Total and by Gender")
    .tab_spanner(
        label="Total",
        columns=["Total", "Gold", "Silver", "Bronze"]
    )
    .tab_spanner(
        label="Men",
        columns=["Gold_Men", "Silver_Men", "Bronze_Men"]
    )
    .tab_spanner(
        label="Women",
        columns=["Gold_Women", "Silver_Women", "Bronze_Women"]
    )
    .tab_spanner(
        label="Mixed",
        columns=["Gold_Mixed", "Silver_Mixed", "Bronze_Mixed"]
    )
    .tab_spanner(
        label="Open",
        columns=["Gold_Open", "Silver_Open", "Bronze_Open"]
    )
    .cols_label(
        Total="All",
        Gold = "🥇",
        Silver="🥈",
        Bronze="🥉",
        Gold_Men= "🥇",
        Silver_Men = "🥈",
        Bronze_Men = "🥉",
        Gold_Women= "🥇",
        Silver_Women = "🥈",
        Bronze_Women = "🥉",
        Gold_Mixed = "🥇",
        Silver_Mixed = "🥈",
        Bronze_Mixed = "🥉",
        Gold_Open = "🥇",
        Silver_Open = "🥈",
        Bronze_Open = "🥉"
    )
    .tab_style(
        style=style.fill(color="#C6BCBC"),
        locations=loc.body(columns=["Total"]),
    )
    .tab_style(
        style=style.fill(color="#fefefe"),
        locations=loc.body(columns=["Gold", "Silver","Bronze"]),
    )
    .tab_style(
        style=style.fill(color="#dbd9e9"),
        locations=loc.body(columns=["Gold_Men", "Silver_Men","Bronze_Men"]),
    )
    .tab_style(
        style=style.fill(color="#ffb6b6"),
        locations=loc.body(columns=["Gold_Women", "Silver_Women","Bronze_Women"]),
    )
    .tab_style(
        style=style.fill(color="#fefefe"),
        locations=loc.body(columns=["Gold_Mixed", "Silver_Mixed","Bronze_Mixed"]),
    )
    .tab_style(
        style=style.fill(color="#efefef"),
        locations=loc.body(columns=["Gold_Open", "Silver_Open","Bronze_Open"]),
    )
)

table

GT(_tbl_data=          Committee  Total  Gold  Silver  Bronze  Gold_Men  Silver_Men  \
0  🇺🇸 United States    126    40      44      42        13          16   
1          🇨🇳 China     91    40      27      24        17          10   
2          🇯🇵 Japan     45    20      12      13        12           7   
3      🇦🇺 Australia     53    18      19      16         5           9   
4         🇫🇷 France     64    16      26      22        11          14   
5    🇳🇱 Netherlands     34    15       7      12         6           2   
6  🇬🇧 Great Britain     65    14      22      29         6          15   
7    🇰🇷 South Korea     32    13       9      10         5           2   
8          🇮🇹 Italy     40    12      13      15         3           9   
9        🇩🇪 Germany     33    12      13       8         4           5   

   Bronze_Men  Gold_Women  Silver_Women  Bronze_Women  Gold_Mixed  \
0          23          26            23            18           1   
1           7          19            15            16           3   
2           4           8             3             7           0   
3           6          13             9             8           0   
4          13           4            11             8           1   
5           3           8             5             8           1   
6           9           6             7            15           0   
7           3           7             5             5           1   
8          11           7             4             4           2   
9           2           3             6             6           1   

   Silver_Mixed  Bronze_Mixed  Gold_Open  Silver_Open  Bronze_Open  
0             3             1          0            2            0  
1             2             1          1            0            0  
2             2             1          0            0            1  
3             0             2          0            1            0  
4             0             0          0            1            1  
5             0             0          0            0            1  
6             0             2          2            0            3  
7             2             2          0            0            0  
8             0             0          0            0            0  
9             1             0          4            1            0  , _body=<great_tables._gt_data.Body object at 0x7aaccda408c0>, _boxhead=Boxhead([ColInfo(var='Committee', type=<ColInfoTypeEnum.stub: 2>, column_label='Committee', column_align='left', column_width=None), ColInfo(var='Total', type=<ColInfoTypeEnum.default: 1>, column_label='All', column_align='right', column_width=None), ColInfo(var='Gold', type=<ColInfoTypeEnum.default: 1>, column_label='🥇', column_align='right', column_width=None), ColInfo(var='Silver', type=<ColInfoTypeEnum.default: 1>, column_label='🥈', column_align='right', column_width=None), ColInfo(var='Bronze', type=<ColInfoTypeEnum.default: 1>, column_label='🥉', column_align='right', column_width=None), ColInfo(var='Gold_Men', type=<ColInfoTypeEnum.default: 1>, column_label='🥇', column_align='right', column_width=None), ColInfo(var='Silver_Men', type=<ColInfoTypeEnum.default: 1>, column_label='🥈', column_align='right', column_width=None), ColInfo(var='Bronze_Men', type=<ColInfoTypeEnum.default: 1>, column_label='🥉', column_align='right', column_width=None), ColInfo(var='Gold_Women', type=<ColInfoTypeEnum.default: 1>, column_label='🥇', column_align='right', column_width=None), ColInfo(var='Silver_Women', type=<ColInfoTypeEnum.default: 1>, column_label='🥈', column_align='right', column_width=None), ColInfo(var='Bronze_Women', type=<ColInfoTypeEnum.default: 1>, column_label='🥉', column_align='right', column_width=None), ColInfo(var='Gold_Mixed', type=<ColInfoTypeEnum.default: 1>, column_label='🥇', column_align='right', column_width=None), ColInfo(var='Silver_Mixed', type=<ColInfoTypeEnum.default: 1>, column_label='🥈', column_align='right', column_width=None), ColInfo(va

In [17]:
table.save("./output/country_medals.png", scale=4)

GT(_tbl_data=          Committee  Total  Gold  Silver  Bronze  Gold_Men  Silver_Men  \
0  🇺🇸 United States    126    40      44      42        13          16   
1          🇨🇳 China     91    40      27      24        17          10   
2          🇯🇵 Japan     45    20      12      13        12           7   
3      🇦🇺 Australia     53    18      19      16         5           9   
4         🇫🇷 France     64    16      26      22        11          14   
5    🇳🇱 Netherlands     34    15       7      12         6           2   
6  🇬🇧 Great Britain     65    14      22      29         6          15   
7    🇰🇷 South Korea     32    13       9      10         5           2   
8          🇮🇹 Italy     40    12      13      15         3           9   
9        🇩🇪 Germany     33    12      13       8         4           5   

   Bronze_Men  Gold_Women  Silver_Women  Bronze_Women  Gold_Mixed  \
0          23          26            23            18           1   
1           7          19            15            16           3   
2           4           8             3             7           0   
3           6          13             9             8           0   
4          13           4            11             8           1   
5           3           8             5             8           1   
6           9           6             7            15           0   
7           3           7             5             5           1   
8          11           7             4             4           2   
9           2           3             6             6           1   

   Silver_Mixed  Bronze_Mixed  Gold_Open  Silver_Open  Bronze_Open  
0             3             1          0            2            0  
1             2             1          1            0            0  
2             2             1          0            0            1  
3             0             2          0            1            0  
4             0             0          0            1            1  
5             0             0          0            0            1  
6             0             2          2            0            3  
7             2             2          0            0            0  
8             0             0          0            0            0  
9             1             0          4            1            0  , _body=<great_tables._gt_data.Body object at 0x7aaccda408c0>, _boxhead=Boxhead([ColInfo(var='Committee', type=<ColInfoTypeEnum.stub: 2>, column_label='Committee', column_align='left', column_width=None), ColInfo(var='Total', type=<ColInfoTypeEnum.default: 1>, column_label='All', column_align='right', column_width=None), ColInfo(var='Gold', type=<ColInfoTypeEnum.default: 1>, column_label='🥇', column_align='right', column_width=None), ColInfo(var='Silver', type=<ColInfoTypeEnum.default: 1>, column_label='🥈', column_align='right', column_width=None), ColInfo(var='Bronze', type=<ColInfoTypeEnum.default: 1>, column_label='🥉', column_align='right', column_width=None), ColInfo(var='Gold_Men', type=<ColInfoTypeEnum.default: 1>, column_label='🥇', column_align='right', column_width=None), ColInfo(var='Silver_Men', type=<ColInfoTypeEnum.default: 1>, column_label='🥈', column_align='right', column_width=None), ColInfo(var='Bronze_Men', type=<ColInfoTypeEnum.default: 1>, column_label='🥉', column_align='right', column_width=None), ColInfo(var='Gold_Women', type=<ColInfoTypeEnum.default: 1>, column_label='🥇', column_align='right', column_width=None), ColInfo(var='Silver_Women', type=<ColInfoTypeEnum.default: 1>, column_label='🥈', column_align='right', column_width=None), ColInfo(var='Bronze_Women', type=<ColInfoTypeEnum.default: 1>, column_label='🥉', column_align='right', column_width=None), ColInfo(var='Gold_Mixed', type=<ColInfoTypeEnum.default: 1>, column_label='🥇', column_align='right', column_width=None), ColInfo(var='Silver_Mixed', type=<ColInfoTypeEnum.default: 1>, column_label='🥈', column_align='right', column_width=None), ColInfo(va

In [19]:
table.write_raw_html("./output/country_medals.html")